In [41]:
import pandas as pd

In [42]:
# #Création de la DataBase
# import sqlite3

# #Connexion base de donnée
# conn = sqlite3.connect('logs_test.db')

# #Curseur pour éxécuter les commandes SQL
# cursor = conn.cursor()

# #Création de la table pour les logs
# cursor.execute(
#     '''
#     CREATE TABLE IF NOT EXISTS logs (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     date TEXT,
#     ipsrc TEXT,
#     ipdst TEXT,
#     proto TEXT,
#     portsrc INTEGER,
#     portdst INTEGER,
#     regle INTEGER,
#     action TEXT,
#     interface_in TEXT,
#     interface_out TEXT,
#     divers INTEGER
#     );
#     '''
# )

# #Commit 
# conn.commit()

# #Insérer les données dans la table
# data = pd.read_parquet("data/log_export.parquet")
# data.to_sql('logs', conn, if_exists='replace', index=False)

# #Ferme la connexion
# conn.close()


In [43]:
# # Connexion à la base de données SQLite
# conn = sqlite3.connect('logs_test.db')
# cursor = conn.cursor()

# query = "SELECT * FROM logs LIMIT 10"
# df = pd.read_sql_query(query, conn)

# conn.close()

In [44]:
data = pd.read_parquet("data/log_export.parquet")

In [45]:
data = data.loc[:1000,:]

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1001 non-null   object 
 1   ipsrc          1001 non-null   object 
 2   ipdst          1001 non-null   object 
 3   proto          1001 non-null   object 
 4   portsrc        1001 non-null   float64
 5   portdst        1001 non-null   float64
 6   regle          1001 non-null   float64
 7   action         1001 non-null   object 
 8   interface_in   1001 non-null   object 
 9   Interface_out  0 non-null      object 
 10  divers         1001 non-null   float64
dtypes: float64(4), object(7)
memory usage: 86.2+ KB


# Prétraitement données 

In [47]:
#Supression des colonnes interface_out et divers 
data = data.drop(columns=['Interface_out', 'divers'])

In [48]:
#Supression des lignes avec des valeurs manquantes
data = data.dropna(axis=0)

In [49]:
#Parse la colonne date 
#Convertion en format date 
data['date'] = pd.to_datetime(data['date'])

#Parse le timestamp
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour
data['minute'] = data['date'].dt.minute
data['second'] = data['date'].dt.second

In [50]:
data = data.drop(columns='date')

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ipsrc         1001 non-null   object 
 1   ipdst         1001 non-null   object 
 2   proto         1001 non-null   object 
 3   portsrc       1001 non-null   float64
 4   portdst       1001 non-null   float64
 5   regle         1001 non-null   float64
 6   action        1001 non-null   object 
 7   interface_in  1001 non-null   object 
 8   year          1001 non-null   int32  
 9   month         1001 non-null   int32  
 10  day           1001 non-null   int32  
 11  hour          1001 non-null   int32  
 12  minute        1001 non-null   int32  
 13  second        1001 non-null   int32  
dtypes: float64(3), int32(6), object(5)
memory usage: 86.2+ KB


In [52]:
data.head()

,ipsrc,ipdst,proto,portsrc,portdst,regle,action,interface_in,year,month,day,hour,minute,second
0,94.102.61.47,159.84.146.99,TCP,52502.0,3178.0,999.0,DENY,eth0,2024,3,20,1,29,24
1,176.111.174.85,159.84.146.99,TCP,48739.0,2231.0,999.0,DENY,eth0,2024,3,20,1,29,25
2,66.249.65.106,159.84.146.99,TCP,50501.0,443.0,1.0,PERMIT,eth0,2024,3,20,1,29,27
3,89.248.163.75,159.84.146.99,TCP,43312.0,8845.0,999.0,DENY,eth0,2024,3,20,1,29,34
4,42.58.163.244,159.84.146.99,TCP,9746.0,23.0,7.0,DENY,eth0,2024,3,20,1,29,38


In [53]:
data.isnull().sum().sum()

np.int64(0)

# Encodage variable IP src / dest / proto / interface_in

In [54]:
from sklearn.preprocessing import LabelEncoder

In [55]:
le = LabelEncoder()

ip_proto_cols = ["ipsrc", "ipdst", "proto", "interface_in"]

for col in ip_proto_cols:
    data[col] = le.fit_transform(data[col])

In [56]:
data.head()

,ipsrc,ipdst,proto,portsrc,portdst,regle,action,interface_in,year,month,day,hour,minute,second
0,163,0,0,52502.0,3178.0,999.0,DENY,0,2024,3,20,1,29,24
1,71,0,0,48739.0,2231.0,999.0,DENY,0,2024,3,20,1,29,25
2,136,0,0,50501.0,443.0,1.0,PERMIT,0,2024,3,20,1,29,27
3,149,0,0,43312.0,8845.0,999.0,DENY,0,2024,3,20,1,29,34
4,112,0,0,9746.0,23.0,7.0,DENY,0,2024,3,20,1,29,38


In [57]:
data.isnull().sum().sum()

np.int64(0)

In [58]:
from sklearn.preprocessing import OneHotEncoder

In [59]:
# Initialisation du OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

# Applique le One Hot Encoder
encoded_cols = ohe.fit_transform(data[['action']])

# Récupère le nom des colonnes encodées
encoded_col_names = ohe.get_feature_names_out(['action'])

# Créer un DataFrame avec les colonnes encodées
encoded_df = pd.DataFrame(encoded_cols, columns=encoded_col_names)
encoded_df.index = data.index

# Concaténer les colonnes encodées avec le DataFrame original
data_encoded = pd.concat([data, encoded_df], axis=1)
data_encoded = data_encoded.drop(columns='action')


In [60]:
encoded_df.shape

(1001, 2)

In [61]:
data_encoded.head()

,ipsrc,ipdst,proto,portsrc,portdst,regle,interface_in,year,month,day,hour,minute,second,action_DENY,action_PERMIT
0,163,0,0,52502.0,3178.0,999.0,0,2024,3,20,1,29,24,1.0,0.0
1,71,0,0,48739.0,2231.0,999.0,0,2024,3,20,1,29,25,1.0,0.0
2,136,0,0,50501.0,443.0,1.0,0,2024,3,20,1,29,27,0.0,1.0
3,149,0,0,43312.0,8845.0,999.0,0,2024,3,20,1,29,34,1.0,0.0
4,112,0,0,9746.0,23.0,7.0,0,2024,3,20,1,29,38,1.0,0.0


In [62]:
data_encoded.isnull().sum().sum()

np.int64(0)

In [63]:
#Standard Scaler variables numériques ????

In [64]:
# #Vider la mémoire du premier data set 
# import gc 

# del data
# gc.collect()

# Modèle Isolation Forest

In [65]:
from sklearn.ensemble import IsolationForest

In [88]:
model_IF = IsolationForest()
pred = model_IF.fit_predict(data_encoded)

In [67]:
anomalies = data_encoded[pred == -1]

In [68]:
len(anomalies)

285

In [69]:
anomalies

,ipsrc,ipdst,proto,portsrc,portdst,regle,interface_in,year,month,day,hour,minute,second,action_DENY,action_PERMIT
0,163,0,0,52502.0,3178.0,999.0,0,2024,3,20,1,29,24,1.0,0.0
1,71,0,0,48739.0,2231.0,999.0,0,2024,3,20,1,29,25,1.0,0.0
3,149,0,0,43312.0,8845.0,999.0,0,2024,3,20,1,29,34,1.0,0.0
4,112,0,0,9746.0,23.0,7.0,0,2024,3,20,1,29,38,1.0,0.0
5,63,0,0,50908.0,443.0,1.0,0,2024,3,20,1,29,39,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,3,0,0,49698.0,51110.0,555.0,0,2024,3,20,2,0,8,0.0,1.0
478,3,0,0,49698.0,51110.0,555.0,0,2024,3,20,2,0,8,0.0,1.0
479,3,0,0,49698.0,51110.0,555.0,0,2024,3,20,2,0,8,0.0,1.0
480,3,0,0,49698.0,51110.0,555.0,0,2024,3,20,2,0,8,0.0,1.0


# Modèle DBSCAN

In [70]:
from sklearn.cluster import DBSCAN

In [71]:
# del anomalies
# del model_IF
# gc.collect()

In [72]:
model_DB = DBSCAN()
model_DB.fit(data_encoded)

DBSCAN()

In [87]:
labels = model_DB.labels_

In [74]:
import numpy as np

In [91]:
cluster = pd.DataFrame(data=labels, columns=['cluster'])

In [90]:
isolation = pd.DataFrame(data=pred, columns=['isolation'])

In [92]:
data_final = pd.concat([data, cluster, isolation], axis=1)

In [93]:
data_final.head()

,ipsrc,ipdst,proto,portsrc,portdst,regle,action,interface_in,year,month,day,hour,minute,second,cluster,isolation
0,163,0,0,52502.0,3178.0,999.0,DENY,0,2024,3,20,1,29,24,-1,-1
1,71,0,0,48739.0,2231.0,999.0,DENY,0,2024,3,20,1,29,25,-1,-1
2,136,0,0,50501.0,443.0,1.0,PERMIT,0,2024,3,20,1,29,27,-1,1
3,149,0,0,43312.0,8845.0,999.0,DENY,0,2024,3,20,1,29,34,-1,-1
4,112,0,0,9746.0,23.0,7.0,DENY,0,2024,3,20,1,29,38,-1,-1


In [94]:
data_final['cluster'].value_counts()

cluster
-1    453
 4    223
 3    175
 5    121
 2     17
 0      7
 1      5
Name: count, dtype: int64

In [99]:
data_final['isolation'].value_counts()

isolation
 1    743
-1    258
Name: count, dtype: int64

In [110]:
pd.crosstab(index=data_final['cluster'],columns=data_final['isolation'])

isolation,-1,1
cluster,,
-1,229,224
0,7,0
1,5,0
2,17,0
3,0,175
4,0,223
5,0,121
